# Proyecto Final Área de Imágenes
### EL5206 Laboratorio de Inteligencia Computacional y Robótica

Integrantes:
* José Díaz
* Luis Jiménez

### Librerías útiles

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### Métricas

Se definen métricas de comparación entre vectores de
características de imágenes, así como medidas para generar
un ranking de imágenes similares a una imagen de consulta
introducida por el usuario.

#### Distancia euclideana

In [2]:
def dist_euc(x, y):

    dist = np.linalg.norm(x-y)
    return dist

#### Distancia chi-cuadrado [1]

In [3]:
def dist_chi2(X, Y):

    chi = 0.5 * np.sum([((x - y) ** 2) / (x + y)
                      for (x, y) in zip(X, Y)])
    return chi

##### Pruebas de dist_euc y dist_chi2

In [ ]:
X = np.array([1,2,3])
Y = np.array([5,6,7])
print(dist_euc(X, Y), dist_chi2(X, Y))

#### Medida de Ranking

In [ ]:
#def rank(image, Nrel):



#### Ranking normalizado

In [ ]:
#def rank_norm(image, N, Nrel):



### Referencias utilizadas

[1] https://www.geeksforgeeks.org/chi-square-distance-in-python/

